In [1]:
#!/usr/bin/env python3
# pip install MDAnalysis

import argparse
import numpy as np
import MDAnalysis as mda

/Users/samuel/Desktop/postdoc_PhLAM/codes/MD_analysis/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- User inputs ---
dump_file = "/Users/samuel/Desktop/postdoc_PhLAM/codes/MD_analysis/Normal_300K_01eV/2/nve_1.lammpstrj"   # LAMMPS dump
data_file = None                # e.g. "lammps.data" or None
manual_unwrap = False           # True = simple bond-free unwrapping (orthorhombic boxes only)

# %%
# --- Load Universe ---
if data_file:
    u = mda.Universe(data_file, dump_file, format="LAMMPSDUMP")
else:
    u = mda.Universe(dump_file, format="LAMMPSDUMP")

n_atoms = u.atoms.n_atoms
n_frames = len(u.trajectory)
print(f"Atoms: {n_atoms}, Frames: {n_frames}")

# %%
# --- Allocate arrays ---
X = np.empty((n_frames, n_atoms, 3), dtype=np.float64)
V = np.empty((n_frames, n_atoms, 3), dtype=np.float64) if u.trajectory.ts.has_velocities else None
T = np.empty(n_frames, dtype=np.float64)
L = np.empty((n_frames, 3), dtype=np.float64)  # box lengths per frame (Å)

# %%
# --- Read frames (no unwrap call) ---
for i, ts in enumerate(u.trajectory):
    X[i] = u.atoms.positions
    if V is not None:
        vel = u.atoms.velocities
        if vel is None:
            if i == 0:
                print("No velocities found; skipping velocity extraction.")
            V = None
        else:
            V[i] = vel
    T[i] = getattr(ts, "time", float(i))
    L[i] = ts.dimensions[:3]  # [lx, ly, lz]

# %%
# --- Optional simple unwrapping without bonds (orthorhombic only) ---
if manual_unwrap:
    # Assumes per-frame displacement < half the box length along each axis.
    # Works for orthorhombic cells. Not valid for triclinic.
    Xu = X.copy()
    for i in range(1, n_frames):
        box = L[i]  # shape (3,)
        box_safe = np.where(box > 0.0, box, 1.0)
        d = Xu[i] - Xu[i-1]
        shift = np.round(d / box_safe)
        Xu[i] -= shift * box_safe
    X = Xu

# %%
# --- Print summary ---
print("\nTrajectory info:")
print(f"  atoms      : {n_atoms}")
print(f"  frames     : {n_frames}")
print(f"  X shape    : {X.shape}")
print(f"  V shape    : {None if V is None else V.shape}")
print(f"  T shape    : {T.shape}")
print(f"  Box (first): {L[0]}")
print("\nExample frame 0:")
print(f"  t = {T[0]:.4f}")
print(f"  pos[0] = {X[0,0]}")
if V is not None:
    print(f"  vel[0] = {V[0,0]}")



/Users/samuel/Desktop/postdoc_PhLAM/codes/MD_analysis/.venv/lib/python3.9/site-packages/MDAnalysis/topology/LAMMPSParser.py:632: UserWarning: Guessed all Masses to 1.0
  warnings.warn('Guessed all Masses to 1.0')
/Users/samuel/Desktop/postdoc_PhLAM/codes/MD_analysis/.venv/lib/python3.9/site-packages/MDAnalysis/coordinates/LAMMPS.py:598: UserWarning: Reader has no dt information, set to 1.0 ps
  ts.data['time'] = step_num * ts.dt


Atoms: 98, Frames: 2701

Trajectory info:
  atoms      : 98
  frames     : 2701
  X shape    : (2701, 98, 3)
  V shape    : (2701, 98, 3)
  T shape    : (2701,)
  Box (first): [ 9.86546421  9.86546421 26.72727394]

Example frame 0:
  t = 0.0000
  pos[0] = [ 1.21243     6.5801301  14.67269993]
  vel[0] = [ 1.28322995  0.25261399 -4.53163004]


In [4]:
print(V[0])

[[  1.28322995   0.25261399  -4.53163004]
 [ -1.12343001  -0.221156   -11.07269955]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.  

In [ ]:
import numpy as np
from lxml import etree

def read_vasprun_velocities(xml_path):
    """Retourne une liste de tableaux (n_atoms,3) pour chaque bloc <varray name='velocities'>."""
    vel_steps, current = [], None
    with open(xml_path, "rb") as fh:
        ctx = etree.iterparse(fh, events=("start","end"), recover=True, huge_tree=True)
        for event, elem in ctx:
            tag = elem.tag.rsplit('}', 1)[-1] if isinstance(elem.tag, str) else ""
            if event == "start" and tag == "varray":
                name = (elem.get("name") or "").lower()
                if name == "velocities":
                    current = []
            elif event == "end" and tag == "v" and current is not None:
                txt = (elem.text or "").strip()
                if txt:
                    try:
                        current.append([float(x) for x in txt.split()])
                    except ValueError:
                        pass
                elem.clear()
            elif event == "end" and tag == "varray":
                if current is not None and current:
                    vel_steps.append(np.array(current, dtype=float))
                current = None
                elem.clear()
            else:
                elem.clear()
    return vel_steps

# Utilisation
vel_list = read_vasprun_velocities("Normal_300K_01eV_vasp_files/vasprun-1_full.xml")  # ou "vasprun.xml.gz"
print(f"Étapes détectées : {len(vel_list)}")
if vel_list:
    print("Forme première étape :", vel_list[0].shape)

    

Étapes détectées : 0


In [31]:
# Count literal occurrences in the file
import re, gzip

file = "Normal_300K_01eV_vasp_files/vasprun-1_full.xml"  # or "vasprun.xml.gz"
  # or "vasprun.xml.gz"
def _read_bytes(p):
    return gzip.open(p, "rb").read() if p.endswith(".gz") else open(p, "rb").read()

b = _read_bytes(file)
print("count <varray name=\"velocities\"> =", len(re.findall(br'<varray\s+name\s*=\s*"velocities"', b)))
print("count <varray name=\"velocity\">   =", len(re.findall(br'<varray\s+name\s*=\s*"velocity"',   b)))

count <varray name="velocities"> = 1
count <varray name="velocity">   = 0
